## HF加载Tokenizer的方法

- transformers库中的加载方法：对tokenizers库的高层封装，提供与模型（如BERT、Llama）无缝集成的接口
- tokenizers库中的加载方法：底层实现，提供更高效的分词（Rust-based）机制，常用于transformers库的快速Tokenizer

### transformers库中的加载方法

#### AutoTokenizer.from_pretrained()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # 自动加载 BertTokenizerFast
inputs = tokenizer("Hello, MageEdu!", return_tensors="pt")  # 输出: {'input_ids': tensor([[101, 7592, 1010, 2088, 999, 102]]), 'attention_mask': ...}
print(inputs)  # 输出{'input_ids': tensor([[  101,  7592,  1010, 17454,  2098,  2226,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
print(tokenizer.decode(inputs["input_ids"][0]))  # " [CLS] hello, MageEdu! [SEP] "

#### 具体Tokenizer类.from_pretrained()

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.add_tokens(["new_token1", "new_token2"])  # 添加自定义标记
inputs = tokenizer("Hello, [new_token1]!", return_tensors="pt")
print(inputs)  # 输出包含新标记的 input_ids
print(tokenizer.decode(inputs["input_ids"][0]))

### tokenizers库中的加载方法

In [ ]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
encoding = tokenizer.encode("Hello, world!")
print(encoding.tokens)  # ['[CLS]', 'hello', ',', 'world', '!', '[SEP]']

## HF加载模型进行任务推理

### 使用模型类进行自动加载

#### AutoModel

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. & 2. 加载分词器和模型
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    # CausalLM模型如GPT-2默认没有pad token，需要手动设置
    tokenizer.pad_token = tokenizer.eos_token

# 3. 预处理：将文本转换为模型可理解的输入张量
input_text = "The future of AI is"
# 使用 __call__ 方法 (更推荐) 编码，并返回PyTorch张量
inputs = tokenizer(input_text, return_tensors='pt')  # 编码并返回PyTorch张量（包括input_ids和attention_mask）
# 从inputs字典中提取出input_ids张量
input_ids = inputs['input_ids']

# 4. 模型推理：获得原始输出（logits）
# 在推理时禁用梯度，节省内存并加速
with torch.no_grad():
    # 使用 generate 方法进行生成，并添加采样策略以提高多样性
    # - do_sample=True: 启用采样
    # - top_k=30: 从概率最高的30个词中采样
    # - temperature=0.9: 控制随机性
    #outputs = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.pad_token_id)  # 使用generate方法进行生成
    #outputs = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.pad_token_id, do_sample=True)
    #outputs = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.pad_token_id, do_sample=True, top_k=30)
    outputs = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.pad_token_id, do_sample=True, top_k=30, temperature=0.9)

# 5. 后处理：将模型输出的token IDs解码回文本
# outputs[0] 是生成的 token ID 序列
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

#### AutoModel指定Device

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- 1. 设备和模型定义 ---
# 自动选择最佳设备（GPU优先，否则CPU）
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"

# --- 2. 加载分词器和模型 ---
# 加载模型时直接将其移动到选定的设备
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# 设置分词器的 padding token，这对批处理很重要
if tokenizer.pad_token is None:
    # CausalLM模型如GPT-2默认没有pad token，需要手动设置
    tokenizer.pad_token = tokenizer.eos_token

# --- 3. 预处理：使用更安全的 __call__ 方法 ---
input_text = "The future of AI is"
# 使用 __call__ 方法 (更推荐) 编码，并返回PyTorch张量
inputs = tokenizer(input_text, return_tensors='pt').to(device)
input_ids = inputs['input_ids']

# --- 4. 模型推理：使用更灵活的 generate 方法 ---
# 在推理时禁用梯度，节省内存并加速
with torch.no_grad():
    # 使用 generate 方法进行生成，并添加采样策略以提高多样性
    # - do_sample=True: 启用采样
    # - top_k=50: 从概率最高的50个词中采样
    # - temperature=0.9: 控制随机性
    outputs = model.generate(
        input_ids,
        max_length=100,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        top_k=50,
        temperature=0.9
    )

# --- 5. 后处理：解码回文本 ---
# outputs[0] 是生成的 token ID 序列
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"输入设备: {device}")
print("---")
print(f"生成的文本:\n{generated_text}")

### Pipeline()

#### 极简示例

最小可运行示例。

In [ ]:
from transformers import pipeline

# 最简单：情感分析（会自动选择默认模型）
classifier = pipeline("sentiment-analysis")
text_input = ["I love MageEdu.", "A terrible day."]

results = classifier(text_input)
print(results)

#### 指定模型

In [ ]:
from transformers import pipeline

# 最简单：情感分析（会自动选择默认模型）
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
text_input = ["I love MageEdu.", "A terrible day."]

results = classifier(text_input)
print(results)

#### 指定模型、设备和深度学习框架的示例

In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model="gpt2",           # 或者本地路径 "./my_model"
    device=0,               # -1 为 CPU，非负 int 表示 CUDA 设备编号
    framework="pt",         # "pt" 或 "tf"（需已安装相应框架）
)

text_input = "Once upon a time"

print(pipe(text_input, max_length=40))

#### 指定模型、设备和深度学习框架的示例2

In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen3-0.6B",           # 或者本地路径 "./my_model"
    device=0,               # -1 为 CPU，非负 int 表示 CUDA 设备编号
    framework="pt",         # "pt" 或 "tf"（需已安装相应框架）
    truncation=True,
)

text_input = "从前有座山，"
results = pipe(text_input, max_length=256)

print(results)

#### 任务的专用参数示例

文本生成（text-generation）的专用参数
- max_length：控制生成文本的最大长度（输入+输出）
- max_new_tokens：控制模型新生成的最大token数量
- do_sample：是否使用采样策略（如设置为 True可生成更多样化的文本，False则为贪婪解码）
- temperature：控制采样随机性，值越高输出越随机，越低则越确定
- top_k：在采样时，仅从概率最高的k个token中选取
- top_p：(nucleus sampling) 仅从累积概率超过p的最小token集合中选取
- num_return_sequences：指定生成多少个不同的序列


In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen3-0.6B",           # 或者本地路径 "./my_model"
    device=0,               # -1 为 CPU，非负 int 表示 CUDA 设备编号
    framework="pt",         # "pt" 或 "tf"（需已安装相应框架）
    truncation=True,
)

text_input = "从前有座山，"
results = pipe(text_input, max_new_tokens=256, top_k=50, temperature=0.8)  # 文本生成任务的专用参数max_new_tokens、top_k和temperature

print(results)

#### 将前面AutoModel的示例改为Pipeline的实现

In [ ]:
from transformers import pipeline
import torch

# --- 1. 设备和模型定义 ---
# 推荐：自动选择最佳设备（GPU优先，否则CPU）
# 注意：pipeline 默认在支持的情况下会使用 GPU，但最好显式指定
if torch.cuda.is_available():
    # 使用第一个 GPU (device=0)
    device = 0 
else:
    # 否则使用 CPU (device=-1)
    device = -1 

model_name = "Qwen/Qwen3-0.6B"
input_text = "欢迎来到马哥教育学习云计算/SRE、大模型、网络安全、云原生、DevOps等课程"

# --- 2. 实例化 pipeline 对象 (一步完成加载、配置和设备分配) ---
generator = pipeline(
    "text-generation", 
    model=model_name, 
    device=device,
)

# --- 3. 调用 pipeline 进行推理 (一步完成预处理、推理和后处理) ---
# 将原始模型推理时的参数直接传递给 pipeline
# Pipeline 内部会将这些参数传给底层的 model.generate() 方法
results = generator(
    input_text,
    max_new_tokens=256,
    do_sample=True,         # 启用采样
    top_k=50,               # 从概率最高的 50 个词中采样
    temperature=0.9,        # 控制随机性
    num_return_sequences=1  # 只返回一个生成的序列
)

# --- 4. 打印最终结果 ---
# Pipeline 返回一个列表，其中包含一个或多个字典
generated_text = results[0]['generated_text']

print(f"使用的设备索引: {device} (0为GPU, -1为CPU)")
print("---")
print(f"生成的文本:\n{generated_text}")

## 数据集

### 数据集基础

#### 安装datasets

```bash
pip install datasets
```

还有两个可选的扩展，若需要，可以安装
```bash
pip install datasets[audio]   # 音频数据集
pip install datasets[vision]  # 图像/视频数据集
```

下面的代码如果能够执行成功，则代码datasets库安装成功，且能够正常加载数据集。

In [ ]:
from datasets import load_dataset;ds = load_dataset('wikitext', 'wikitext-2-raw-v1');print(ds['train'], ds['validation'], ds['test'])

#### 加载数据集

In [ ]:
from datasets import load_dataset

# 加载整个数据集（通常返回包含train、test等split的DatasetDict）
dataset_dict = load_dataset("imdb")
print(dataset_dict['train'][0])     # 访问训练集第一条数据

# 加载特定子集切分
train_dataset = load_dataset("imdb", split="train")
print(f"\nimdb训练集:\n {train_dataset}")

# 加载数据集的特定配置和部分数据
wikitext_103_subset = load_dataset('wikitext', 'wikitext-2-raw-v1', split="train[:100]")
print(f"\nwikitext_103训练集的前100行:\n {wikitext_103_subset}")

#### 流式加载超大数据集

In [ ]:
streamed_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split="train", streaming=True)

# 流式数据集可迭代，并可结合skip, take, shuffle等进行操作
for example in streamed_dataset.take(10):  # 只取前10个样本
    print(example)

#### 基本信息与结构检查

In [ ]:
from datasets import load_dataset

# 加载数据集
dataset_dict = load_dataset("imdb")

# 1. 查看DatasetDict的键（ splits）
print("数据划分:", list(dataset_dict.keys()))

# 2. 获取训练集Dataset对象
train_dataset = dataset_dict["train"]

# 3. 查看数据集形状（样本数, 列数）
print("数据集形状 (行, 列):", train_dataset.shape)

# 4. 查看列名
print("列名:", train_dataset.column_names)

# 5. 查看特征详情
print("特征详情:", train_dataset.features)

# 6. 查看样本数量
print("训练集样本数:", train_dataset.num_rows)

#### 数据访问与预览

In [ ]:
from datasets import load_dataset
import random
import pandas as pd # 引入 pandas 确保 to_pandas 可用

# 1. 加载 IMDb 数据集（情感分析）
# load_dataset默认返回DatasetDict，我们选择'train'切分进行操作
raw_datasets = load_dataset("imdb")
dataset = raw_datasets['train']

print(f"--- 成功加载 IMDb 训练集，包含 {len(dataset)} 条记录 ---")
print(f"--- dataset 变量类型检查: {type(dataset)} ---\n") 
# 确保 dataset 是 <class 'datasets.arrow_dataset.Dataset'>

# 1. 索引访问 (dataset[0])
print("1. 索引访问 (dataset[0]): 查看第一个样本")
example_0 = dataset[0]
print(f"评论 (text): {example_0['text'][:70]}...")
print(f"标签 (label): {example_0['label']} (0=负面, 1=正面)\n")

# 2. 切片访问 (dataset[:3])
# 直接切片操作，返回一个新的 Dataset 对象
first_three_examples = dataset[:3]
print("2. 切片访问 (dataset[:3]): 查看前三个样本")
print(f"类型: {type(first_three_examples)}")
print(f"第三条评论: {first_three_examples['text'][2][:50]}...\n")

# 3. 访问单列 (dataset['column_name'][index])
print("3. 访问单列 (dataset['label'][:5]): 查看前五个样本的标签")
labels_list = dataset['label'][:5]
print(f"前五个标签: {labels_list}\n")

# 4. dataset.select(indices)
random_indices = random.sample(range(len(dataset)), 3)
selected_subset = dataset.select(random_indices)

print(f"4. dataset.select({random_indices}): 查看随机选择的 3 个样本")
print(f"用于选择的索引: {random_indices}") 
print(f"第一个随机样本评论: {selected_subset['text'][0][:50]}...\n")


# 5. dataset.to_pandas()
# 使用 select(range(10)) 获取前10条记录的子集，确保它是一个Dataset对象
df_subset = dataset.select(range(10)) 
df = df_subset.to_pandas()

print("5. dataset.to_pandas() (前 10 条): 转换为DataFrame便于表格化预览")
print(f"子集类型检查: {type(df_subset)}") # 再次检查子集类型
print(df[['text', 'label']])
print("\n")

# 6. dataset.data.table
arrow_table = dataset.data.table

print("6. dataset.data.table: 访问底层 Arrow 表")
print(f"Arrow 表的 Schema: {arrow_table.schema}")
# 仅展示 Arrow 头部信息
print(f"前 2 条 Arrow 数据: {arrow_table.slice(0, 2).to_pydict()['text']}")

#### 数据处理与转换 - map()

In [ ]:
from datasets import load_dataset

# 加载IMDB数据集示例
dataset = load_dataset("imdb", split="train[:10]")  # 取前10条样本

#-----------------
# 处理单个样本
#-----------------

# 示例1: 将文本转换为小写
def lowercase_function(example):
    example["text"] = example["text"].lower()
    return example

dataset_lower = dataset.map(lowercase_function)
print(dataset_lower[0]["text"])  # 将第1行输出为小写文本

# 示例2: 添加新特征（文本长度）
def add_text_length(example):
    example["text_length"] = len(example["text"].split())  # 计算词数
    return example

dataset_with_length = dataset.map(add_text_length)
print(dataset_with_length[0]["text_length"])  # 输出文本长度

#--------------------
# 处理批次样本
#--------------------
# 示例3: 批量文本小写化
def lowercase_batch(batch):
    batch["text"] = [text.lower() for text in batch["text"]]
    return batch

dataset_lower_batch = dataset.map(lowercase_batch, batched=True)

# 示例4: 批量添加文本长度
def add_length_batch(batch):
    batch["text_length"] = [len(text.split()) for text in batch["text"]]
    return batch

dataset_with_length_batch = dataset.map(add_length_batch, batched=True, batch_size=500)  # 自定义批次大小

# 示例5: 使用Tokenizer进行批量分词 (常见且高效)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(batch):
    # tokenizer会自动处理列表输入
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset.column_names)  # 查看新增的input_ids, attention_mask等列


#### 数据处理与转换 - filter()

In [ ]:
from datasets import load_dataset

# 加载IMDB数据集示例
dataset = load_dataset("imdb", split="train")   # train子集中的全部样本

# 示例1： 保留文本长度超过100个字符的评论
long_reviews = dataset.filter(lambda example: len(example["text"]) > 100)
print("满足条件的行数（文本长度超过100个字符）：", len(long_reviews))
print(f"第一个样本评论: {long_reviews[0]['text']}")

# 示例2： 仅保留正面评论（假设标签label为 1 代表正面）
positive_reviews = dataset.filter(lambda example: example["label"] == 1)
print("满足条件的行数（正面评论）：", len(positive_reviews))

# 示例3： 多条件过滤，保留正面且文本较长的评论
positive_long_reviews = dataset.filter(
    lambda example: example["label"] == 1 and len(example["text"]) > 500
)
print("满足条件的行数（正面评论且文本长度大于500）：", len(positive_long_reviews))

# 示例4： 根据文本内容筛选（包含特定关键词）
# 筛选出评论中包含特定关键词的样本，例如包含 "disaster"（灾难）的评论
# 定义筛选函数：文本中包含 "disaster" 关键词
def contains_keyword(example):
    # 使用 Python 的 in 操作符进行字符串查找（不区分大小写）
    return "disaster" in example['text'].lower()

disaster_dataset = dataset.filter(contains_keyword)
print(f"筛选后的样本数（包含disaster关键词）: {len(disaster_dataset)}")
print(f"第一个样本评论: {disaster_dataset[0]['text']}")

#### 数据集拆分与合并

将IMDB的原始训练集拆分为训练集和验证集。

In [ ]:
from datasets import load_dataset, DatasetDict

#----------------------
# 数据集拆分示例
#----------------------

# 1. 加载IMDb训练集 (25000行)
dataset = load_dataset("imdb", split='train')

# 2. 将数据集划分为80%训练集和20%验证集
split_dict = dataset.train_test_split(
    test_size=0.2, 
    seed=42,
    # 假设需要分层抽样以保持 label (0/1) 比例一致
    stratify_by_column='label' 
)

# 3. 结果是一个 DatasetDict
train_ds = split_dict['train']   # 20000 行
val_ds = split_dict['test']      # 5000 行

print(f"训练集大小: {len(train_ds)}")
print(f"验证集大小: {len(val_ds)}")

#----------------------
# 数据集合并示例
#----------------------
from datasets import concatenate_datasets, load_dataset

# 4. 模拟两个具有相同结构的子集
ds_part1 = dataset.select(range(10)) # 通过指定索引列表，从数据集中精确选择一个或多个样本子集（前10行）
ds_part2 = dataset.select(range(10, 20)) # 10-19 行

print(f"Part 1 大小: {len(ds_part1)}")
print(f"Part 2 大小: {len(ds_part2)}")

# 5. 按行合并 (axis=0)
merged_dataset = concatenate_datasets([ds_part1, ds_part2])

print(f"\n按行合并后的总大小: {len(merged_dataset)}") # 20 行

# 6. 验证合并结果（第一个样本来自 Part 1, 第 11 个样本来自 Part 2）
print(f"第一个样本的 text: {merged_dataset[0]['text'][:30]}...")
print(f"第十一个样本的 text: {merged_dataset[10]['text'][:30]}...")

#### 列操作

In [ ]:
from datasets import load_dataset, Value

# 1. 加载数据
dataset = load_dataset("imdb", split="train[:1000]") # 取前1000条作为示例
print("数据集原有列的列名:", dataset.column_names)
print("数据集原有列的Features:", dataset.features)

# 2. 重命名列（标准化）
dataset = dataset.rename_column("text", "review")
print("改字字段名称后（text->review）的数据集列名:", dataset.column_names)

# 3. 添加新特征列（文本长度）
text_lengths = [len(review.split()) for review in dataset["review"]]
dataset = dataset.add_column("review_length", text_lengths)
print("添加列后的数据集列名:", dataset.column_names)
print("各个列的Features（含新增列review_length）:", dataset.features)

# 4. 转换新列的数据类型
new_features = dataset.features.copy()
new_features["review_length"] = Value("int32") # 使用更节省空间的int32
dataset = dataset.cast(new_features)
print("新增列（review_length）Features修改后的结果:", dataset.features)

# 5. 最终移除不必要的中间列（或原始列）
# 假设我们只需要清洗后的评论和标签
dataset = dataset.remove_columns(["review_length"]) # 移除临时列

print("最终数据集列名:", dataset.column_names)
print("最终第一条数据:", dataset[0])

#### 动态格式设定

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# 1. 加载数据集
ds = load_dataset("imdb", split="train[:100]") # 取前100条样本以便快速演示

# 2. 加载分词器 (例如，使用BERT的分词器)
model_name = "bert-base-uncased" # 可选: "distilbert-base-uncased", "roberta-base" 等
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. 定义分词函数
def tokenize_function(examples):
    # 对文本进行分词处理。返回一个包含'input_ids', 'attention_mask'等的字典
    return tokenizer(
        examples["text"], 
        padding=True, # 填充到批次内最大长度，更高效，也可用 "max_length" 填充到固定长度
        truncation=True, # 截断到模型最大长度
        max_length=512, # 指定最大长度（例如512）
        # return_tensors="pt"  # 通常不在map中设置，用set_format统一转换更高效
    )

# 4. 应用分词函数到整个数据集
# batched=True 可以显著加速处理
tokenized_ds = ds.map(tokenize_function, batched=True)

# 5. 查看处理后的数据集特征（确认新增了分词器产生的列）
# 注意其中的attention_mask是填充掩码，不是因果掩码
print(f"处理后的数据集特征：{tokenized_ds.column_names}")  # 通常输出: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

# 6. 移除原始文本列（如果不需要），以节省内存和显存
tokenized_ds = tokenized_ds.remove_columns(["text"])

# 7. 设置数据集格式，转换为PyTorch张量，并指定需要的列
tokenized_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"] # 选择模型需要的列
)

# 8. 创建DataLoader
dataloader = DataLoader(
    tokenized_ds, 
    batch_size=8, 
    shuffle=True, # 训练时通常打乱数据
    # num_workers=4,  # 可选：使用多进程加载数据以加速（Windows下有时需设为0）
    # pin_memory=True # 可选：如果使用GPU，设置为True可以加速数据从CPU到GPU的传输
)  

# 9. 迭代DataLoader
for batch in dataloader:
    # batch 是一个字典，键是列名，值是对应的张量（形状为 [batch_size, ...]）
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["label"]
    
    # 接下来，通常在这里将批次数据送入模型进行前向传播
    # outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    # loss = outputs.loss
    # ... 其余训练步骤（计算损失、反向传播、优化器步进等）
    
    # 以下是打印信息以供调试和理解
    print(f"Input ID的形状: {input_ids.shape}")
    print(f"注意力掩码的开关：: {attention_mask.shape}")
    print(f"标签: {labels}")
    
    # 通常在这里执行训练循环，这里只是一个示例，所以只运行一个批次就退出
    break # 移除这个 break 来遍历整个数据集

## Evalute库

### 基础使用

#### Metric API基础

**核心是两个方法：load()和compute()**
- evaluate.load(metric_name, **kwargs)​：用于加载指定的评估指标，常用参数
  - metric_name（必选）：指标名称，如 "accuracy"、"f1"、"bleu"等
  - module_type：指定评估类型，默认为 'metric'，可选 'comparison'或 'measurement'，用于防止名称冲突
  - cache_dir：缓存路径
  - keep_in_memory：是否将数据保留在内存中
- metric.compute(predictions, references, **kwargs)：传入预测值和参考值，计算指标结果

In [ ]:
import evaluate

# 1. 加载准确率指标
accuracy_metric = evaluate.load("accuracy")

# 2. 准备数据：真实标签（references）和模型预测标签（predictions）
references = [0, 1, 0, 1, 1]  # 真实标签
predictions = [0, 1, 1, 1, 1]  # 模型预测的标签

# 3. 计算准确率
results = accuracy_metric.compute(references=references, predictions=predictions)

# 4. 查看结果
print(results)  # 输出：{'accuracy': 0.8}

#### Evaluate的典型工作流

In [ ]:
import evaluate

# 1. 加载指标
metric = evaluate.load("accuracy")

# 2. 循环添加批次
for batch in dataloader:
    preds = model(batch["input"])
    metric.add_batch(predictions=preds, references=batch["label"])

# 3. 计算总结果
final_score = metric.compute()
print(final_score)

#### Evaluate与Trainer集成示例

## Trainer

In [ ]:
# 导入必要的库
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate  # 用于评估指标
import numpy as np
import torch
import os

# 0. 设置训练时要使用的GPU，请注意要对照实际的硬件环境进行修改
# 设置环境变量，只允许脚本看到并使用系统中的 GPU 0，以避免自动启用DPP（Jupyter环境不支持）
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
print(f"可见GPU数: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# 1. 加载IMDB数据集与tokenizer
# IMDB数据集包含50000条电影评论，标记为正面或负面情感
dataset = load_dataset("imdb")  # 加载IMDB数据集
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # 加载BERT的tokenizer

# 数据预处理函数：将文本转换为模型可接受的输入格式
def preprocess(examples):
    # 使用tokenizer对文本进行编码，包括截断和填充到最大长度
    return tokenizer(
        examples["text"],           # IMDB数据集中文本字段名为"text"
        truncation=True,            # 截断超过模型最大长度的文本
        padding="max_length",       # 填充到最大长度以确保统一尺寸
        max_length=256              # 设置最大序列长度为256
    )

# 对数据集进行批量预处理
encoded_dataset = dataset.map(preprocess, batched=True)

# 2. 加载预训练模型
# 使用BERT模型进行序列分类，IMDB是二分类任务（正面/负面）
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2  # 设置分类标签数为2
)

# 3. 加载评估指标
# 使用准确率作为评估指标 
metric = evaluate.load("accuracy")

# 定义计算指标的函数，用于训练过程中的评估
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # 解包预测结果和真实标签
    preds = logits.argmax(-1)   # 取logits中最大值的位置作为预测结果
    return metric.compute(predictions=preds, references=labels)  # 计算准确率

# 4. 配置训练参数
training_args = TrainingArguments(
    output_dir="./results",          # 输出目录，用于保存模型和结果
    eval_strategy="epoch",          # 每轮结束后进行评估
    save_strategy="epoch",          # 每轮结束后保存模型
    load_best_model_at_end=True,    # 训练结束后加载最佳模型
    metric_for_best_model="accuracy",  # 用于选择最佳模型的指标
    greater_is_better=True,         # 准确率越高越好
    learning_rate=2e-5,             # 学习率，常用BERT微调的学习率
    per_device_train_batch_size=16, # 每个设备的训练批量大小
    per_device_eval_batch_size=16,  # 每个设备的评估批量大小
    num_train_epochs=3,             # 训练轮数
    weight_decay=0.01,              # 权重衰减，防止过拟合
)

# 创建Trainer实例，用于管理整个训练过程
trainer = Trainer(
    model=model,                     # 要训练的模型
    args=training_args,              # 训练参数
    train_dataset=encoded_dataset["train"],        # 训练集
    eval_dataset=encoded_dataset["test"],          # 评估集（使用测试集作为验证）
    tokenizer=tokenizer,             # tokenizer
    compute_metrics=compute_metrics  # 计算评估指标的函数
)

# 5. 启动训练与评估
trainer.train()  # 开始训练模型

# 训练完成后进行评估
eval_result = trainer.evaluate()
print(f"评估结果: {eval_result}")  # 打印评估结果

# 可选：保存最终模型
trainer.save_model("./imdb_sentiment_model")

### 训练示例

运行下面的训练脚本，需要事先安装transformers、datasets、torch、numpy、evaluate、accelerate等库。

另外，在一些特定的Hugging Face环境中（尤其是较新的transformers版本），即使accelerate版本符合要求，也需要安装accelerate的PyTorch后端依赖，或者transformers在导入时，没有找到accelerate库中所需的某些特定组件或路径。

更常见的原因是，系统环境中安装了多个版本的PyTorch或accelerate，或者Python路径（sys.path）存在问题，导致transformers导入时找不到正确的accelerate安装。因此，如果遇到了无法找到accelerate的问题，可运行如下命令进行重新安装来解决问题。

```bash
pip install transformers[torch] --upgrade
# 或者 确保 accelerate 被正确链接：
pip install accelerate --upgrade
```

如果有必要，可以通过如下代码先检查python环境。
```python
import accelerate
import transformers
print(accelerate.__version__)
print(transformers.__version__)
print(accelerate.__file__) # 确保路径正确
```

下面是一个Evaluate与Trainer集成的示例。

In [ ]:
import torch
import numpy as np
from datasets import load_dataset             # 用于加载 Hugging Face 数据集
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer
)
import evaluate                             # Hugging Face 官方评估库，用于加载指标


# 2. 数据准备与预处理
# 定义使用的模型名称和超参数
MODEL_NAME = "bert-base-uncased" # 选择一个 BERT 风格模型
MAX_LENGTH = 128                 # 序列最大长度
TRAIN_SAMPLE_PERCENT = 5         # 仅使用 1% 的训练数据进行快速演示
EVAL_SAMPLE_PERCENT = 5          # 仅使用 1% 的测试数据进行评估

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 加载 IMDB 数据集，并按百分比截取
print(f"--- 正在加载数据集 (仅使用 {TRAIN_SAMPLE_PERCENT}% 训练集和 {EVAL_SAMPLE_PERCENT}% 测试集) ---")
# 标签列名为 'label'，文本列名为 'text'
raw_train_datasets = load_dataset("imdb", split=f"train[:{TRAIN_SAMPLE_PERCENT}%]")
raw_eval_datasets = load_dataset("imdb", split=f"test[:{EVAL_SAMPLE_PERCENT}%]")

# 定义预处理（分词）函数
def tokenize_function(examples):
    """将文本转换为模型所需的 token ID 和注意力掩码"""
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

# 对数据集应用预处理（并行处理）
tokenized_train_datasets = raw_train_datasets.map(tokenize_function, batched=True)
tokenized_eval_datasets = raw_eval_datasets.map(tokenize_function, batched=True)

# 将原始标签列 'label' 重命名为 'labels'，这是 Trainer 默认识别的标签列名
tokenized_train_datasets = tokenized_train_datasets.rename_column("label", "labels")
tokenized_eval_datasets = tokenized_eval_datasets.rename_column("label", "labels")

# 移除原始文本列，只保留模型训练需要的 columns: input_ids, token_type_ids, attention_mask, labels
tokenized_train_datasets = tokenized_train_datasets.remove_columns(["text"])
tokenized_eval_datasets = tokenized_eval_datasets.remove_columns(["text"])

# 最终训练和评估数据集
train_dataset = tokenized_train_datasets
eval_dataset = tokenized_eval_datasets

# 3. 模型加载与指标函数定义 (Evaluate 库功能演示)
# 3.1 加载模型：使用 AutoModelForSequenceClassification 进行二分类任务
NUM_LABELS = 2
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

# 3.2 加载评估指标：使用 evaluate 库加载准确率 (accuracy) 指标
metric = evaluate.load("accuracy")

def compute_metrics(p):
    """
    Trainer 在评估时会调用此函数。
    输入 p 是一个 EvalPrediction 对象 (包含 predictions 和 label_ids)
    """
    # 提取模型的 logits 或其他预测结果
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    # 转换为最终的类别预测 ID
    predictions = np.argmax(logits, axis=1)
    
    # 使用 evaluate 库计算指标
    return metric.compute(predictions=predictions, references=p.label_ids)

# 4. 实例化 Trainer 并训练 (Trainer 库核心功能演示)
# 4.1 定义训练参数 (TrainingArguments)
OUTPUT_DIR = "./results_trainer_demo"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,                   # 检查点和输出的目录
    num_train_epochs=1,                      # 训练周期数，设置为 1 快速完成
    per_device_train_batch_size=8,           # 每个设备上的训练批次大小
    per_device_eval_batch_size=8,            # 每个设备上的评估批次大小
    warmup_steps=100,                        # 学习率预热步数
    weight_decay=0.01,                       # 权重衰减
    logging_dir='./logs_trainer_demo',       # TensorBoard 日志目录
    logging_steps=50,
    eval_strategy="epoch",                   # 评估策略: 每个 epoch 结束时进行评估
    save_strategy="epoch",                   # 保存策略: 每个 epoch 结束时保存模型
    load_best_model_at_end=True,             # 训练结束后加载基于 metric_for_best_model 的最佳模型
    metric_for_best_model="accuracy",        # 定义衡量最佳模型的指标
    report_to="none",                        # 关闭外部报告工具以保持简洁
)

# 4.2 实例化 Trainer
trainer = Trainer(
    model=model,                             # 传入加载的模型
    args=training_args,                      # 传入训练参数
    train_dataset=train_dataset,             # 传入处理后的训练集
    eval_dataset=eval_dataset,               # 传入处理后的评估集
    tokenizer=tokenizer,                     # 传入分词器（用于数据整理和保存）
    compute_metrics=compute_metrics,         # 传入指标计算函数
)

# 4.3 开始训练
print("\n" + "="*50)
print("--- 启动模型训练 (Trainer.train()) ---")
print("="*50)
train_result = trainer.train()

# 5. 评估和结果打印
# 5.1 记录训练统计信息（如总步数、总时间）
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state() # 保存 Trainer 状态

# 5.2 使用 Trainer.evaluate() 进行最终评估
print("\n" + "="*50)
print("--- 启动模型评估 (Trainer.evaluate()) ---")
print("="*50)
eval_results = trainer.evaluate(eval_dataset=eval_dataset)

# 5.3 打印评估结果，包括由 compute_metrics 计算的准确率
print("\n最终评估结果:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# 6. 模型保存（可选）
# 保存最终模型和分词器
FINAL_SAVE_PATH = f"{OUTPUT_DIR}/final_model"
print(f"\n--- 保存最终模型到: {FINAL_SAVE_PATH} ---")
trainer.save_model(FINAL_SAVE_PATH)
tokenizer.save_pretrained(FINAL_SAVE_PATH)

print("\n--- 任务流程已完成 ---")